In [ ]:
import pandas as pd
from openai import OpenAI
from datetime import datetime
import json
import os
from dotenv import load_dotenv #load the key from .env

In [ ]:
#Load from .env file if exists
load_dotenv()

#Read from environment
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

Cookie Sheet processing

In [ ]:
import time

# Load template and input Excel files
#template = pd.ExcelFile('template.xlsx')
input_file_name = "APM0004066 - Deloitte Assistant - Cookies_Assistant.xlsx"
output_file_name = "Done-APM0004066 - Deloitte Assistant - Cookies_Assistant.xlsx"
input_data = pd.read_excel(input_file_name, sheet_name=None)  # read all sheets into a dict of DataFrames
print("File extraction done")

# Process Cookies sheet
try:
    cookies_df = input_data.get('Cookies')
    #Ensure required columns exist
    for col in ['Category', 'Description']:
          if col not in cookies_df.columns:
                 cookies_df[col] = '' #Add new column if not exist
    print(cookies_df.head())
except:
    print("The document does not have cookie sheet")

cookies_df['Description'] = cookies_df['Description'].astype(str)
cookies_df['Category'] = cookies_df['Category'].astype(str)

# Iterate through each row in Cookies sheet
for i, row in cookies_df.iterrows():
    name = row['Name']  # get the cookie name
    expiration = row['Expiration']  # get expiration value
    lifespan = row['Lifespan']  # get lifespan value
    thirdparty = row['ThirdParty'] # get Thirdparty value
    setby = row['Set By'] # get the setby value
    domain = row['Domain'] # get the domain name
    cookies_df.columns = cookies_df.columns.str.strip()

    if thirdparty == "" or (domain == cookies_df['Domain'].iloc[0] or domain.endswith("."+cookies_df['Domain'].iloc[0])):
         cookies_df.at[i, 'ThirdParty'] = 'FALSE'
    else:
         cookies_df.at[i, 'ThirdParty'] = 'TRUE'
    
    #if row['Set By'] =='nan':
    if pd.isna(row['Set By']):
         cookies_df.at[i, 'Set By'] = "https://"+domain

    # Check and format expiration date or lifespan
    if pd.isnull(expiration) or str(expiration).strip().lower()=='session':
        cookies_df.at[i, 'Expiration'] = 'Infinity'
        cookies_df.at[i, 'Lifespan'] = 'Session'
    elif expiration == 'Infinity':
        # If expiration is Infinity, set lifespan as Session
        cookies_df.at[i, 'Lifespan'] = 'Session'  
    elif lifespan == 'Session':
        # If lifespan is Session, set the expiration as Infinity
        cookies_df.at[i, 'Expiration'] = 'Infinity'
    elif pd.notnull(expiration) and expiration != 'Infinity': 
        # Format valid date to required ISO format
        expiration = str(expiration).strip()  # remove spaces
        expiration = expiration.replace('/', '-')  # replace slashes with dashes if needed
        expiration = pd.to_datetime(expiration,errors='coerce').strftime('%Y-%m-%dT%H:%M:%S.000Z')
        print("log:",expiration)
        cookies_df.at[i, 'Expiration'] = expiration
        cookies_df.at[i, 'Lifespan'] = 'Permanent'

# Call ChatGPT to categorize cookie and generate description
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a cookie categorizer."},
            {"role": "user", "content": f"Return JSON object with keys 'Category' and 'Description' for cookie name {name} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {name}"}],
        temperature=0 )
    text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
    print("GPT:",text)
    result = json.loads(text)  # parse JSON
    cookies_df.at[i, 'Category'] = result.get('Category', '').strip()
    cookies_df.at[i, 'Description'] = result.get('Description', '').strip()
    sucess = True #done!
    print(f"Got the category done!")

import openpyxl
# Find lowest expiration timestamp

# Extract the expiration dates after 2000 year
cookies_df['Expiration'] = pd.to_datetime(cookies_df['Expiration'],errors = 'coerce')
valid_dates = cookies_df[cookies_df['Expiration'].dt.year>=2000]['Expiration']
print(f"all dates:{valid_dates}")

# Find the lowest year to find the cookie scan date
min_date = valid_dates.min()
print(f"Minimum date is:{min_date}")

# convert the lowest date into iSO standards
if pd.notna(min_date):
     manual_scan_date = min_date.strftime('%Y-%m-%dT%H:%M:%S.000Z')
     print(f"manual_scan_date is:{manual_scan_date}")
else:
     manual_scan_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S.000Z')
     print(f"min_date is NaT. Cannot format, so using current date {manual_scan_date}")
    
# Format the expiration date format, only for the dates.
cookies_df['Expiration'] = cookies_df['Expiration'].where(
    cookies_df['Expiration'].apply(lambda x: isinstance(x,pd.Timestamp)), pd.NaT
).apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S.000Z") if pd.notna(x) else 'Infinity')

# print("DataFrame shape:",cookies_df.shape)
# print("Column names:",cookies_df.columns.tolist())
# print("Last 5 rows:\n",cookies_df.head())

#Avoid duplicating footer rows and Append Timestamp and Website info
if not cookies_df.iloc[-3:, 0].str.contains("Time stamp of Manual Scan run").any():     
    cookies_df.loc[len(cookies_df)] = [''] * len(cookies_df.columns)     
    cookies_df.loc[len(cookies_df)] = ['Time stamp of Manual Scan run', manual_scan_date] + [''] * (len(cookies_df.columns) - 2)     
    domain_value = cookies_df['Domain'].iloc[0] if 'Domain' in cookies_df.columns else ''     
    cookies_df.loc[len(cookies_df)] = ['Website', domain_value] + [''] * (len(cookies_df.columns) - 2) 

# Convert all to string before writing to Excel 
cookies_df = cookies_df.astype(str)

# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl',mode='a', if_sheet_exists ='replace') as writer:
    cookies_df.to_excel(writer, sheet_name='Cookies', index=False)

print(f"Done,{output_file_name}")

Local Sheet processing

In [ ]:
# Process Local sheet
try:
    local_df = input_data.get('Local Storage')
    
    for col in ['Category', 'Description']:
        if col not in local_df.columns:
            local_df[col] = '' #Add new column if not exist
    
    print(local_df.head())
except:
    print("There is no Local Storage sheet exist")
#Ensure required columns exist

# Process Local Storage sheet
local_df = input_data.get('Local Storage')
if local_df is not None:
    for i, row in local_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Respond with only valid JSON object with keys 'Category' and 'Description' for local storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}. the respond must be without code blocks, markdown, or extra quotes."}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        local_df.at[i, 'Category'] = result.get('Category', '').strip()
        local_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for local storage")

# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    local_df.to_excel(writer, sheet_name='Local Storage', index=False)

print("Done!")

Session Storage Sheet processing

In [ ]:
# Process Cookies sheet
try:
    session_df = input_data.get('Session Storage')
    
    for col in ['Category', 'Description']:
        if col not in session_df.columns:
            session_df[col] = '' #Add new column if not exist
    
    print(session_df.head())
except:
    print("There is no Session Storage sheet exist")
#Ensure required columns exist

# Process Session Storage sheet
session_df = input_data.get('Session Storage')
if session_df is not None:
    for i, row in session_df.iterrows():
        key = row['Key']  # key name
 
        # Call ChatGPT to get category and description
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cookie categorizer."},
                {"role": "user", "content": f"Respond with only valid JSON object with keys 'Category' and 'Description' for Session storage cookie name {key} choose the category from: 'Strictly Necessary Cookies', 'Advertising and Targetting Cookie's, 'Functional Cookies', 'Analytics and Performance Cookies' and description value must be short one line for the web cookie named {key}. the respond must be without code blocks, markdown, or extra quotes."}],
            temperature=0 )
        text = response.choices[0].message.content.strip()  # remove any extra spaces/newlines
        print("GPT:",text)
        result = json.loads(text)  # parse JSON
        session_df.at[i, 'Category'] = result.get('Category', '').strip()
        session_df.at[i, 'Description'] = result.get('Description', '').strip()
        sucess = True #done!
        print(f"Got the category done for Session storage")

# Write updated data to new Excel files
with pd.ExcelWriter(output_file_name,engine='openpyxl', mode='a', if_sheet_exists ='replace') as writer:
    session_df.to_excel(writer, sheet_name='Session Storage', index=False)

print("Done!")  